In [1]:
# Packages
install.packages("feather")

# Librairies
library(aws.s3)
library(data.table)
library(feather)

# Environnement
bucket_name <- "maeldieudonne"

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



<h1>Résultats électoraux</h1>

plm = 0-1-2 (0 : communes non PLM ; 1 : 45 arrondissements PLM ; 2 : 3 communes entières PLM)  
codecommune2 = code de la nouvelle commune de rattachement ou de substitution (au cas statut1999=2,3,4) ou de la commune actuelle (au cas statut1999=1). Le fichier comprend plusieurs lignes par codecommune2 dès lors que plusieurs communes ont fusionné en une seule entre 1943 et 1999.  
dep2 = code du nouveau département de rattachement ou de substitution (au cas statut1999=2,3,4) ou du département actuel (au cas statut1999=1)

In [2]:
object_key <- "/diffusion/Elections_pres/pres1981comm.feather"
pres1981 <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
pres1981 <- as.data.table(pres1981)

In [3]:
# Recodages
candidats <- fread("candidats.csv", header = TRUE)

## Participation
pres1981[, part := votants/inscrits]
pres1981[, partT2 := votantsT2/inscritsT2]

## Droite / gauche
T1mask <- candidats[DrGau == "G", .(variable = paste0("voix", Candidat))]
pres1981[, GD_ratio_T1 := rowSums(.SD, na.rm = TRUE) / exprimes, .SDcols = T1mask$variable]

T2mask <- candidats[DrGau == "G", .(variable = paste0("voixT2", Candidat))]
T2candidates <- intersect(T2mask$variable, names(pres1981))
pres1981[, GD_ratio_T2 := rowSums(.SD, na.rm = TRUE) / exprimesT2, .SDcols = T2candidates]

## Libéralisme culturel / conservatisme moral
T1mask <- candidats[LibCons == "L", .(variable = paste0("voix", Candidat))]
pres1981[, lib_ratio_T1 := rowSums(.SD, na.rm = TRUE) / exprimes, .SDcols = T1mask$variable]

In [4]:
# Nettoyage
prefixes_to_drop <- c("exprimes", "inscrits", "nom", "pervote", "ppar", "pvoix", "pvote", "voix", "vot")
pres1981 <- pres1981[, .SD, .SDcols = setdiff(names(pres1981), grep(paste0("^", paste(prefixes_to_drop, collapse = "|")), names(pres1981), value = TRUE))]
pres1981 <- pres1981[, -c("nuls", "plmdoublon", "perpar", "year")]

<h1>Propriété immobilière</h1>

npropri81 = nombre de ménages propriétaires dans la commune  
nlogement1981 = nombre total de logements dans la commune (propriétaires, locataires ou logés gratuitement) (résidences principales)  
ppropri1981 = proportion de ménages propriétaires de leur logement dans la commune

In [5]:
object_key <- "/diffusion/Proprietaires/proprietairescommunes.feather"
propri <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
propri <- as.data.table(propri)

<h1>Démographie et géographie</h1>
<h2>1. Population</h2>

pop1981  
popagglo1981

In [6]:
object_key <- "/diffusion/Taille_agglo_commune/popcommunes.feather"
popul <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
popul <- as.data.table(popul)

<h2>2. Âge</h2>

age1981 = âge moyen de la commune  
prop0141981 = proportion de 0-14 ans dans la commune  
prop15391981 = proportion de 15-39 ans dans la commune  
prop40591981 = proportion de 40-59 ans dans la commune  
prop60p1981 = proportion de 60 ans et + dans la commune

In [7]:
object_key <- "/diffusion/Age_csp/agesexcommunes.feather"
ages <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
ages <- as.data.table(ages)

<h1>Formation, diplômes, religion </h1>

<h2>Diplômes</h2>

nodip1981 : nombre total de personnes 25+ sans diplôme ou dont le diplôme le plus élevé est le BEPC, le brevet des collèges, le BEP ou le CAP  
bac1981: nombre total de personnes 25+ dont le diplôme le plus élevé est le baccalauréat général ou technologique  
sup1981 : nombre total de personnes 25+ diplômées du supérieur  

In [8]:
object_key <- "/diffusion/Diplomes/diplomescommunes.feather"
diplo <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
diplo <- as.data.table(diplo)

# Raporter nodip1981, bac1981 et sup1981 au nombre d'habitants.

<h2>Religiosité</h2>

prefract1791 = proportion de prêtres réfractaires en 1791  
nclerge1791 = nombre total des prêtres soumis au serment de 1791

pclerge1856 : effectifs totaux du clergé (prêtres, religieux, religieuses) exprimés en proportion de la population totale  
pmessalisants1950 : effectifs totaux de personnes allant à la messe du dimanche en 1950 en proportion de la population totale

In [9]:
object_key <- "/diffusion/Enseignement_prive/religiositecommunes.feather"
religcomm <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
religcomm <- as.data.table(religcomm)

# Recodages
# nclerge1791 à rapporter à la taille de la population pour avoir une idée de la présence des institutions religieuses dans la commune.

In [10]:
object_key <- "/diffusion/Enseignement_prive/religiositedepartements.feather"
religdep <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
religdep <- as.data.table(religdep)

<h1>CSP, emplois, professions</h1>

pagri1981 = proportion d’agriculteurs parmi les actifs 25-54 ans (=agri/pact)  
pindp1981 = proportion d’indépendants (artisans, commerçants, chefs d’entreprises) parmi les actifs 25-54 ans (=indp/pact)  
pcadr1981 = proportion de cadres et professions intellectuelles supérieures parmi les actifs 25-54 ans (=cadr/pact)  
ppint1981 = proportion de professions intermédiaires parmi les actifs 25-54 ans (=pint/pact)  
pempl1981 = proportions d’employés parmi les actifs 25-54 ans (=empl/pact)  
pouvr1981 = proportion d’ouvriers parmi les actifs 25-54 ans (=ouvr/pact)  
pchom1981 = proportion de chômeurs parmi les actifs 25-54 ans (=chom/pact)

In [11]:
object_key <- "/diffusion/CSP/cspcommunes.feather"
catsociopro <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
catsociopro <- as.data.table(catsociopro)

<h1>Nationalités</h1>

petranger1981 = proportion de personnes de nationalité étrangère

In [12]:
object_key <- "/diffusion/Nationalites/etrangerscommunes.feather"
nation <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
nation <- as.data.table(nation)

<h1>Productions et revenus</h1>

revmoy1981 = revenu moyen par habitant de la commune (exprimé en euros de 2022)

In [13]:
object_key <- "/diffusion/Revenus/revcommunes.feather"
reven <- aws.s3::s3read_using(
  FUN = arrow::read_feather,
  object = object_key,
  bucket = bucket_name,
  opts = list("region" = "")
)
reven <- as.data.table(reven)

<h1>Construction de la base</h1>

In [14]:
pres1981[propri, c("ppropri1981") :=
             .(i.ppropri1981), on = "codecommune"]
pres1981[popul, c("pop1981", "popagglo1981") :=
             .(i.pop1981, i.popagglo1981), on = "codecommune"]
pres1981[ages, c("age1981", "prop0141981", "prop15391981", "prop40591981", "prop60p1981") :=
             .(i.age1981, i.prop0141981, i.prop15391981, i.prop40591981, i.prop60p1981), on = "codecommune"]
# Diplômes
pres1981[religcomm, c("prefract1791") :=
             .(i.prefract1791), on = "codecommune"]
pres1981[religdep, c("pclerge1856", "pmessalisants1950") :=
             .(i.pclerge1856, i.pmessalisants1950), on = "dep"]
pres1981[catsociopro, c("pagri1981", "pindp1981", "pcadr1981", "ppint1981", "pempl1981", "pouvr1981", "pchom1981") :=
             .(i.pagri1981, i.pindp1981, i.pcadr1981, i.ppint1981, i.pempl1981, i.pouvr1981, i.pchom1981), on = "codecommune"]
pres1981[nation, c("petranger1981") :=
             .(i.petranger1981), on = "codecommune"]
pres1981[reven, c("revmoy1981") :=
             .(i.revmoy1981), on = "codecommune"]

In [15]:
write_feather(pres1981, "pres1981.feather")

In [17]:
print(names(pres1981))

 [1] "dep"               "codecommune"       "plm"              
 [4] "abs"               "nulsT2"            "absT2"            
 [7] "codecommune2"      "dep2"              "part"             
[10] "partT2"            "GD_ratio_T1"       "GD_ratio_T2"      
[13] "lib_ratio_T1"      "ppropri1981"       "pop1981"          
[16] "popagglo1981"      "age1981"           "prop0141981"      
[19] "prop15391981"      "prop40591981"      "prop60p1981"      
[22] "prefract1791"      "pclerge1856"       "pmessalisants1950"
[25] "pagri1981"         "pindp1981"         "pcadr1981"        
[28] "ppint1981"         "pempl1981"         "pouvr1981"        
[31] "pchom1981"         "petranger1981"     "revmoy1981"       
